#Create Agents to Research and Write an Article

 This project demonstrates the use of multi-agent systems via the CrewAI framework to automate the process of generating insightful and engaging articles.

# Install required packages

In [ ]:

!pip install langchain langchain_core langchain_groq crewai duckduckgo-search crewai[tools]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.1/66.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

# Import necessary libraries and modules

In [ ]:

import os
import json
from typing import Union, List, Tuple, Dict
from datetime import datetime
from random import randint

# Import necessary classes and functions

In [ ]:

from langchain_core.agents import AgentFinish
from langchain.schema import AgentFinish
import langchain_groq
from langchain_groq import ChatGroq
from crewai import Crew, Agent, Task, Process
from langchain_community.tools import DuckDuckGoSearchRun
from crewai_tools import tool

Set your Groq API key:

In [ ]:
import os
from google.colab import userdata

os.environ["GROQ_API_KEY"] = "your API Key"


In [ ]:
# Initialize the Groq language model

GROQ_LLM = ChatGroq(
    api_key=os.getenv("GROQ_API_KEY"),
    model="llama3-70b-8192"
)

# Define the DuckDuckGo search tool using crewai_tools
@tool('DuckDuckGoSearch')
def search(search_query: str):
    """Search the web for information on a given topic"""
    return DuckDuckGoSearchRun().run(search_query)

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

In [ ]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	  verbose=True,
    llm=GROQ_LLM,
    tools=[search],


)
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm=GROQ_LLM,

)
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm=GROQ_LLM,
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

In [ ]:

plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)


In [ ]:
# Callback Function
# Function to print agent output
def print_agent_output(step_output, crew_name):
    print(f"{crew_name} step output: {step_output}")

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.

In [ ]:
# Create the Crew with agents and tasks
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2,
    step_callback=lambda x: print_agent_output(x, "Crew"),
)


## Running the Crew
**Note**: LLMs can provide different outputs for the same input, so what you get might be different than what you see in this notebook.


In [ ]:
# Define the topic and kickoff the Crew tasks
topic = "AI Agents"
result = crew.kickoff(inputs={"topic": topic})
print(result)


 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on AI Agents.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I need to prioritize the latest trends, key players, and noteworthy news on AI Agents.

Action: DuckDuckGoSearch
Action Input: {"search_query": "AI Agents trends"} 

Here are some important current AI trends to look out for in the coming year. Reality check: more realistic expectations. Multimodal AI. Small (er) language models and open source advancements. GPU shortages and cloud costs. Model optimization is getting more accessible. Customized local models and data pipelines. Google just gave us a tantalizing glimpse into the future of AI agents. Hugh Langley. May 14

This comprehensive article, generated by these agents, demonstrates the power and capabilities of a multi-agent system.

### Navigating GPU Shortages and Cloud Costs

However, the increasing demand for AI agents has led to GPU shortages and skyrocketing cloud costs. This has forced companies to re-evaluate their AI strategies and explore more cost-effective solutions. By doing so, businesses can ensure sustainable AI adoption.

### Model Optimization and Customized Local Models: Tailoring AI to Business Needs

Fortunately, model optimization is becoming more accessible, enabling businesses to develop customized local models and data pipelines tailored to their specific needs. This tailored approach can lead to more efficient AI adoption and better ROI.

### Google's Vision for the Future of AI Agents: Human-Like Interactions

Google's vision for AI agents involves creating more intuitive and human-like interactions. With advancements in multimodal AI and small language models, we can expect AI agents to become even more integrated into our daily lives. This increased integration has the potential to revolutionize the way we work and interact.

**Key Players in AI Agents**
---------------------------

### Google: A Pioneer in AI Agent Development

Google has been at the forefront of AI agent development, leveraging its vast resources and expertise to push the boundaries of AI capabilities. Google's advancements have paved the way for more sophisticated AI agents that can transform industries.

### Other Notable Companies and Researchers: Driving Innovation

Other notable companies and researchers, such as IBM Watson Studio, are also making significant contributions to the field of AI agents. These innovators are driving the development of more advanced AI agents that can tackle complex tasks.

**Noteworthy News and Developments in AI Agents**
---------------------------------------------

### 2023 Gartner Hype Cycle for Artificial Intelligence (AI): AI Agents in the Spotlight

The 2023 Gartner Hype Cycle for Artificial Intelligence (AI) highlights the growing interest in AI agents and their potential to transform industries. This increased interest is a testament to the potential of AI agents to drive business growth.

### AI Agents' Potential to Communicate with Other AI Agents: Seamless Automation

The ability of AI agents to communicate with each other has significant implications for businesses, enabling more efficient decision-making and automation. This seamless automation can lead to increased productivity and reduced costs.

### Growth in the AI Agents Space Heading into 2024: A New Era of Automation

As we head into 2024, we can expect to see continued growth in the AI agents space, with more businesses integrating AI agents into their marketing strategies. This growth is a testament to the potential of AI agents to revolutionize the way businesses operate.

**Applications of AI Agents in Business**
---------------------------------------

### Personalized Marketing Campaigns: Tailoring to Individual Customers

AI agents can help businesses create personalized marketing campaigns tailored to individual customers, increasing engagement and conversion rates. By leveraging AI agents, businesses can create more targeted marketing strategies that drive results.

### AI-Powered Market Research Tools: Data-Driven Decision-Making

AI-powered market research tools can provide businesses with valuable insights, enabling them to make data-driven decisions and stay ahead of the competition. By leveraging AI agents, businesses can make more informed decisions that drive growth.

### Integrating AI Automation into Marketing Strategies: Streamlining Processes

By integrating AI automation into their marketing strategies, businesses can streamline processes, reduce costs, and improve overall efficiency. This increased efficiency can lead to increased productivity and better ROI.

**Conclusion**
----------

AI agents are poised to revolutionize the way businesses operate, and it's essential for marketers, business owners, and AI enthusiasts to stay informed about the latest developments in this field. By understanding the trends, key players, and applications of AI agents, businesses can unlock new opportunities for growth and stay ahead of the competition. Stay informed, and explore ways to integrate AI agents into your business strategy today!

